In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Кернелы, к которым я обращалась за вдохновением:

https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction?scriptVersionId=5301226&cellId=16
https://www.kaggle.com/code/dariyakharytonova/notebook721481d097/

In [2]:
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

**Для начала изучим таблицы с данными, описание столбцов в файлике HomeCredit_columns_description, схему связи БД**

### Train data

In [3]:
train_data = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
pd.set_option('display.max_columns', None)
train_data.head()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/home-credit-default-risk/application_train.csv'

* Это основной датасет, с которым нам предстоит работать. Содержит много столбцов, в т.ч. категориальные переменные, которые, вероятнее всего, нам придется раскладывать на фиктивные переменные. 

* Включает аппликационные переменные соцдем характера, как правило, это не очень сильные переменные в разрезе предсказания кредитоспособности, однако мы всё же попробуем

* Данные, касающиеся дат (рождения, опыта работы) требуют интерпретации для лучшего понимания

* Содержит много сведений про билдинг, где живёт клиент, которые, вероятнее всего, не описывают целевую переменную

> Зависимая переменная TARGET качественная, представляет собой дефолт флаг: TARGET = 1 - кредитополучатель не погасит кредит, TARGET = 0 - всё будет ок

### Bureau

In [ ]:
bureau_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")
bureau_balance.head()

In [ ]:
bureau = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
bureau.head()

* содержит переменные CREDIT_DAY_OVERDUE, AMT_CREDIT_MAX_OVERDUE, CNT_CREDIT_PROLONG, AMT_CREDIT_SUM_DEBT, AMT_CREDIT_SUM_OVERDUE, которые хорошо характеризуют платежное поведение клиента и могут быть нам интересны с точки зрения предсказания выхода в дефолт
* однако качество данных из кредитного регистра может быть не очень хорошим: содержать много пропущенных и неотформатированных значений

> 

### POS_CASH balance

In [ ]:
POS_CASH_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")
POS_CASH_balance.head()

* Таблица содержит полезные данные по days past due, описывающие поведение клиента
* Это наши данные(Home Credit), поэтому качество данных, вероятно, будет хорошим

### Previous Applications

In [ ]:
previous_application = pd.read_csv("/kaggle/input/home-credit-default-risk/previous_application.csv")
previous_application.head()

* NAME_CONTRACT_TYPE, NAME_PORTFOLIO можно использовать для классификации наших сделок (лучше вторую, тк более ёмкие и унифицированные названия)
* AMT_CREDIT, AMT_DOWN_PAYMENT пригодятся в качестве справочной инфы
* NAME_CONTRACT_STATUS, CODE_REJECT_REASON также могут быть полезны 
* NAME_PRODUCT_TYPE - кросс-селам доверяем больше, так как это уже наши клиенты, в которых мы уверены настолько, что предложили им ещё одну сделку
* DAYS_FIRST_DUE, DAYS_LAST_DUE_1ST_VERSION, DAYS_LAST_DUE проверим в качестве предикторов

In [ ]:
credit_card_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/credit_card_balance.csv")
credit_card_balance.head()

* Таблица содержит в себе в т.ч. транзакционные данные, которые могут явиться хорошими предикторами кредитоспособности или на их основании можно вывести сильные переменные

### Installment Payments

In [ ]:
installments_payments = pd.read_csv("/kaggle/input/home-credit-default-risk/installments_payments.csv")
installments_payments.head()

* Данные по платежам содержащиеся в таблице являются поведенческими, могут быть полезны для нас позже

### Test

In [ ]:
test_data = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
test_data.head()

> то же, что и Train, но без целевой переменной

 **Train - основной сет, который мы обработаем и на котором построим базовую модель. В последующих версиях с целью улучшения модели можно будет подтягивать переменные из других табличек**

# Предобработка Train 

In [ ]:
train_data.head()

## Missings

In [ ]:
missings = train_data.isnull()
missings

In [ ]:
for column in missings.columns.values.tolist():
    print(column)
    print (missings[column].value_counts())
    print("")

Если переменная содержит более 5-10% пропущенных значений, имеет смысл её удалить и не учитывать в дальнейшем анализе. Если исходя из бизнес-логики переменная с пропущенными значениями может хорошо предсказывать таргет или пропуски объяснимы, можно попытаться её "спасти" и догрузить данные из альтернативного источника. 

В нашем случае большинство переменных с большим кол-вом миссингов характеризуют жилплощадь кредитополучателя, ээкспертным путём определим, что их можно удалить. Маловероятно, что, например, материал стен позволит нам предсказать целевую переменную.

Переменные AMT_REQ_CREDIT_BUREAU_.. имеют одинаковое кол-во пропущенных значений, предположим, это можно объяснить, пока не будем их отбрасывать

OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE похожие по смыслу переменные, тоже имеют одинаковое количество пропусков, эти переменные сохраним для дальнейшего анализа

In [ ]:
train_data = train_data.drop(['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_AVG', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_MEDI',
       'LIVINGAPARTMENTS_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI',
       'FLOORSMIN_AVG', 'YEARS_BUILD_MODE', 'YEARS_BUILD_MEDI',
       'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_AVG', 'LANDAREA_MEDI',
       'LANDAREA_MODE', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG',
       'BASEMENTAREA_MODE', 'NONLIVINGAREA_MEDI',
       'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'ELEVATORS_MEDI',
       'ELEVATORS_MODE', 'ELEVATORS_AVG', 'WALLSMATERIAL_MODE',
       'APARTMENTS_MODE', 'APARTMENTS_MEDI', 'APARTMENTS_AVG',
       'ENTRANCES_MODE', 'ENTRANCES_AVG', 'ENTRANCES_MEDI', 'LIVINGAREA_MEDI',
       'LIVINGAREA_MODE', 'LIVINGAREA_AVG', 'HOUSETYPE_MODE', 'FLOORSMAX_MEDI',
       'FLOORSMAX_AVG', 'FLOORSMAX_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'TOTALAREA_MODE', 'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE'], axis = 1)
train_data

In [ ]:
test_data = test_data.drop(['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_AVG', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_MEDI',
       'LIVINGAPARTMENTS_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI',
       'FLOORSMIN_AVG', 'YEARS_BUILD_MODE', 'YEARS_BUILD_MEDI',
       'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_AVG', 'LANDAREA_MEDI',
       'LANDAREA_MODE', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG',
       'BASEMENTAREA_MODE', 'NONLIVINGAREA_MEDI',
       'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'ELEVATORS_MEDI',
       'ELEVATORS_MODE', 'ELEVATORS_AVG', 'WALLSMATERIAL_MODE',
       'APARTMENTS_MODE', 'APARTMENTS_MEDI', 'APARTMENTS_AVG',
       'ENTRANCES_MODE', 'ENTRANCES_AVG', 'ENTRANCES_MEDI', 'LIVINGAREA_MEDI',
       'LIVINGAREA_MODE', 'LIVINGAREA_AVG', 'HOUSETYPE_MODE', 'FLOORSMAX_MEDI',
       'FLOORSMAX_AVG', 'FLOORSMAX_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'TOTALAREA_MODE', 'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE'], axis = 1)
test_data.head()

In [ ]:
# https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction?scriptVersionId=5301226&cellId=16

def missing_values_table(df):

        mis_val = df.isnull().sum()
        
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # таблица с рез-тами
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        return mis_val_table_ren_columns

In [ ]:
missing_values = missing_values_table(train_data)
missing_values.loc[missing_values['% of Total Values'] > 5].index

In [ ]:
missing_values = missing_values_table(train_data)
missing_values

In [ ]:
# столбцы имеют не так много пропусков, но и не несут полезной информации

train_data = train_data.drop(['NAME_TYPE_SUITE', 'AMT_GOODS_PRICE'], axis = 1)

In [ ]:
test_data = test_data.drop(['NAME_TYPE_SUITE', 'AMT_GOODS_PRICE'], axis = 1)

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(train_data.dtypes)

In [ ]:
# Типы данных оставшихся переменных, имеющих пропущенные значения:


# EXT_SOURCE_2 - float64, заменим средним
# AMT_ANNUITY - float64, заменим средним
# CNT_FAM_MEMBERS - float64, заменим наиболее распростаненным значением
# DAYS_LAST_PHONE_CHANGE - float64, заменим средним

In [ ]:
train_data[['EXT_SOURCE_2', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE']]

In [ ]:
# заменим Nan в EXT_SOURCE_2 средним значением
mean_ext_src_2 = train_data["EXT_SOURCE_2"].astype("float").mean(axis = 0)
train_data['EXT_SOURCE_2'].replace(np.nan, mean_ext_src_2, inplace = True)

In [ ]:
# заменим Nan в AMT_ANNUITY средним значением
mean_annuity = train_data["AMT_ANNUITY"].astype("float").mean(axis = 0)
train_data['AMT_ANNUITY'].replace(np.nan, mean_annuity, inplace = True)

In [ ]:
# заменим Nan в DAYS_LAST_PHONE_CHANGE средним значением
mean_days = train_data["DAYS_LAST_PHONE_CHANGE"].astype("float").mean(axis = 0)
train_data['DAYS_LAST_PHONE_CHANGE'].replace(np.nan, mean_days, inplace = True)

In [ ]:
train_data['CNT_FAM_MEMBERS'].value_counts()

In [ ]:
train_data['CNT_FAM_MEMBERS'].replace(np.nan, 2.0, inplace = True)

## Форматирование данных

In [ ]:
train_data.head()

Для удобства восприятия колонки, отображающие кол-во дней, сделаем положительными числами

In [ ]:
train_data['DAYS_BIRTH'] = abs(train_data['DAYS_BIRTH'])
test_data['DAYS_BIRTH'] = abs(test_data['DAYS_BIRTH'])

In [ ]:
train_data['DAYS_EMPLOYED'] = abs(train_data['DAYS_EMPLOYED'])
test_data['DAYS_EMPLOYED'] = abs(test_data['DAYS_EMPLOYED'])

In [ ]:
train_data['DAYS_REGISTRATION'] = abs(train_data['DAYS_REGISTRATION'])
test_data['DAYS_REGISTRATION'] = abs(test_data['DAYS_REGISTRATION'])

In [ ]:
train_data['DAYS_ID_PUBLISH'] = abs(train_data['DAYS_ID_PUBLISH'])
test_data['DAYS_ID_PUBLISH'] = abs(test_data['DAYS_ID_PUBLISH'])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(train_data.dtypes)

Типы данных не требуют корректировок

## Нормализация данных

## Создание фиктивных переменных

Будем использовать метод get_dummies

In [ ]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

print('train_data shape: ', train_data.shape)
print('test_data shape: ', test_data.shape)

Для тренировочного датасета создалось больше переменных, чем для тестового. Чтобы это исправить, используем align  https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction?scriptVersionId=5301226&cellId=29

In [ ]:
train_labels = train_data['TARGET']

train_data, test_data = train_data.align(test_data, join = 'inner', axis = 1)

# выносим целевую переменную в отдельный сет
train_data['TARGET'] = train_labels

print('Training Features shape: ', train_data.shape)
print('Testing Features shape: ', test_data.shape)

In [ ]:
train_data.describe()

# Корреляция между признаками

с целью выбрать признаки, объясняющие нашу целевую переменную, воспользуемся коэффициентом корреляции Пирсона. Для переменных с высокими модулями зависимости относительно остальных посмотрим и P-value. Примем уровень значимрсти равным 5%

In [ ]:
correlations = train_data.corr()['TARGET'].sort_values()


print('Прямая зависимость:\n', correlations.tail(15))
print('\nОбратная зависимость:\n', correlations.head(15))

среди переменных не выявлено сильной зависимости с target (модуль коэффициента Пирсона довольно близок к нулю), это можем объяснить тем, что при использовании неповеденческих характеристик велика доля случайности. 

Проверим подробнее и проанализируем на предмет включения в модель признаки EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1, DAYS_EMPLOYED, DAYS_BIRTH 

### DAYS_BIRTH

In [ ]:

pearson_coef, p_value = stats.pearsonr(train_data['DAYS_BIRTH'], \
                                       train_data['TARGET'])

In [ ]:
pearson_coef, p_value

с p_value всё ок, наблюдается обратная зависимость между возрастом и TARGET = 1. То есть чем старше клиент, тем исправнее он платит.
Визуализируем зависимость переменной и TARGET при помощи kde, чтобы выявить, насколько хорошо различаются выборки

In [ ]:
# KDE для выплаченных займов
sns.kdeplot(train_data.loc[train_data['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE для невыплаченных займов
sns.kdeplot(train_data.loc[train_data['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

plt.xlabel('Age '); plt.ylabel('Density');

In [ ]:
plt.boxplot(train_data[train_data['TARGET'] == 0]['DAYS_BIRTH'] / 365)
plt.show()
print((train_data[train_data['TARGET'] == 0]['DAYS_BIRTH'] / 365).describe())


plt.boxplot(train_data[train_data['TARGET'] == 1]['DAYS_BIRTH'] / 365)
plt.show()
print((train_data[train_data['TARGET'] == 1]['DAYS_BIRTH'] / 365).describe())

Действительно, видим, что значение медианы возраста недобросовестных кредитоплательщиков меньше. 

### DAYS_EMPLOYED

Несмотря на то, что кэф корреляции с таргет невысокий, я бы всё равно проверила эту фичу, так как зачастую большой опыт работы сотрудника положительно коррелирует с исправными выплатами по обязательствам.

In [ ]:
pearson_coef, p_value = stats.pearsonr(train_data['DAYS_EMPLOYED'], \
                                       train_data['TARGET'])
pearson_coef, p_value

Переменная значима, визуализируем её по классам

In [ ]:
plt.boxplot(train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED'].dropna())
plt.show()
print((train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED'].dropna()).describe())


plt.boxplot(train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED'].dropna())
plt.show()
print((train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED'].dropna()).describe())

Обнаружены выбросы! Почистим их

In [ ]:
(train_data['DAYS_EMPLOYED'] ).describe()

In [ ]:
# Значение 365243.000000 заменим NaN, затем высчитаем медиану в переменной без выбросов и 
# заменим NaN правильной медианой

train_data['DAYS_EMPLOYED_ANOM'] = 1*(train_data['DAYS_EMPLOYED'] == 365243)
test_data['DAYS_EMPLOYED_ANOM'] = 1*(test_data['DAYS_EMPLOYED'] == 365243)
train_data["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
test_data["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

In [ ]:
(train_data['DAYS_EMPLOYED']).describe()

In [ ]:
# заменим Nan в DAYS_EMPLOYED средним значением
mean_seniority = train_data["DAYS_EMPLOYED"].astype("float").mean(axis = 0)
train_data['DAYS_EMPLOYED'].replace(np.nan, mean_seniority, inplace = True)

In [ ]:
# заменим Nan в DAYS_EMPLOYED средним значением
mean_seniority = test_data["DAYS_EMPLOYED"].astype("float").mean(axis = 0)
test_data['DAYS_EMPLOYED'].replace(np.nan, mean_seniority, inplace = True)

In [ ]:
(train_data['DAYS_EMPLOYED'] / 365).describe()
plt.boxplot((train_data['DAYS_EMPLOYED'] / 365).dropna())
plt.show()

Теперь выбросов нет. Сразу откорректируем тестовый датасет

In [ ]:
(test_data['DAYS_EMPLOYED'] / 365).describe()
plt.boxplot((test_data['DAYS_EMPLOYED'] / 365).dropna())
plt.show()

Вернемся к визуализации зависимости целевой переменной от опыта работы

In [ ]:
plt.boxplot(train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED'].dropna() / 365, vert = False)
plt.show()
print((train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED'].dropna()/ 365).describe())

plt.boxplot(train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED'].dropna()/ 365, vert = False)
plt.show()
print((train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED'].dropna()/ 365).describe())

Мы подправили качество данных, однако от этого переменная не стала заметно различать более опытных от менее отпытных. Попробуем произвести следующую модификацию: найти отношение опыта работы к возрасту.

In [ ]:
train_data['DAYS_EMPLOYED_FRAC'] = train_data['DAYS_EMPLOYED'] / train_data['DAYS_BIRTH']
test_data['DAYS_EMPLOYED_FRAC'] = test_data['DAYS_EMPLOYED'] / test_data['DAYS_BIRTH']
train_data.head()

In [ ]:
plt.boxplot(train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED_FRAC'].dropna(), vert=False)
plt.show()
print(train_data[train_data['TARGET'] == 0]['DAYS_EMPLOYED_FRAC'].describe())


plt.boxplot(train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED_FRAC'].dropna(), vert=False)
plt.show()
print(train_data[train_data['TARGET'] == 1]['DAYS_EMPLOYED_FRAC'].describe())

In [ ]:
pearson_coef, p_value = stats.pearsonr(train_data['DAYS_EMPLOYED_FRAC'], \
                                       train_data['TARGET'])
pearson_coef, p_value

Переменная стала чуть сильнее, чем DAYS_EMPLOYED, однако возраст всё же более сильная фича

### EXT_SOURCES

In [ ]:
ext_sources_stats = train_data[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].corr()

In [ ]:
ext_sources_stats

Видим, что признаки коррелируют с целевой переменной, и видим автокорреляцию между ними. Она вполне объяснима: переменные однотипны. Это может повлиять на дальнейшее кач-во модели, скоро мы об этом узнаем.

# Моделирование

В задачах с кредитным скорингом чаще всего используются алгоритмы классификации: регрессия, дерево решений. Модель, построенная на логистической регрессии, будет обладать чуть меньшей силой, однако будет более долговечной и не будет требовать частого внесения изменений, особенно при значительном изменении внешних факторов. Если речь о банке, то эти плюсы имеют особый вес, ведь там внедрение новых моделей происходит не так просто и быстро. 

Однако сейчас наша цель - построить максимально сильную модель, чтобы получить максимальный результат! Поэтому используем desicion tree ;)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score



In [ ]:
if 'TARGET' in train_data:
    train_labels = train_data['TARGET']
    train_data = train_data.drop(columns=['TARGET'])
    
# train_modified = train.drop(columns=['SK_ID_CURR'])
feature_names = list(train_data.columns)
test_id = test_data['SK_ID_CURR']
# test_modified = test.drop(columns=['SK_ID_CURR'])
test_id

Уберем все пропущенные значения, если что-то ещё не убрали

In [ ]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(train_data)
train_data = imputer.transform(train_data)
test_data = imputer.transform(test_data)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, random_state = 42)

Ищем оптимальные параметры модели:

In [ ]:
import sys 
from sklearn.model_selection import GridSearchCV, KFold 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# params = {'n_estimators': [32, 64, 128, 256, 512]}
# gs_clf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), params)
# gs_clf.fit(x_train, y_train)
# print(gs_clf.best_params_)

Попробуем несколько видов моделей с разным кол-вом estimators

In [ ]:
# clf64 = RandomForestClassifier(n_estimators = 64, random_state=42, n_jobs=-1)
# clf64.fit(x_train, y_train)

In [ ]:
# pred_train = clf64.predict_proba(x_train)[:, 1]
# pred_val = clf64.predict_proba(x_val)[:, 1]
# print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
# print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

In [ ]:
clf128 = RandomForestClassifier(n_estimators = 128, random_state = 42, n_jobs = -1)
clf128.fit(x_train, y_train)

pred_train = clf128.predict_proba(x_train)[:, 1]
pred_val = clf128.predict_proba(x_val)[:, 1]
print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

In [ ]:
# clf256 = RandomForestClassifier(n_estimators = 256, random_state = 42, n_jobs = -1)
# clf256.fit(x_train, y_train)

# pred_train = clf256.predict_proba(x_train)[:, 1]
# pred_val = clf256.predict_proba(x_val)[:, 1]
# print('ROC-AUC train', roc_auc_score(y_train.values, pred_train))
# print('ROC-AUC validation', roc_auc_score(y_val.values, pred_val))

Мы хотим, чтобы площадь под ROC-кривой была равна единице, поэтому оставим n_estimators = 128

In [ ]:
feature_importance = pd.DataFrame({'feature': feature_names,
                                   'importance': clf128.feature_importances_})

In [ ]:
feature_importance.sort_values(by = 'importance', ascending = False).head(10)

EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, DAYS_EMPLOYED_FRAC,  DAYS_EMPLOYED, которые мы анализировали ранее, действительно имеют влияние на целевую переменную

In [ ]:
test_pred = clf128.predict_proba(test_data)[:, 1]

sub = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': test_pred})
sub.to_csv('./Tayas_submission.csv', index = False)